# 睡姿展示系统

这是一个基于Web前端的睡姿压力图展示系统，支持五类睡姿（仰卧、俯卧、左侧卧、右侧卧、坐姿）的热力图可视化和用户切换功能。

## 1. 导入必要的库和模块

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import warnings
warnings.filterwarnings('ignore')

# 导入自定义模块
import sys
sys.path.append('/workspaces/codespaces-jupyter/project')
from dataloader.get_data import build_dataset, get_person_folders, load_pressure_data, compute_average_pressure_map

# 设置matplotlib中文字体
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("所有库导入完成！")

## 2. 数据加载和预处理

In [ ]:
# 数据路径设置
data_root = "/workspaces/codespaces-jupyter/project/data/text_data"

# 扩展get_data模块以支持坐姿（标签7）
def get_extended_label_mapping():
    """获取扩展的标签映射关系，包括坐姿"""
    return {
        1: 1,  # 标准仰卧 -> 仰卧
        2: 2,  # 标准俯卧 -> 俯卧  
        3: 3,  # 左侧卧（弯腿) -> 左侧卧
        4: 3,  # 左侧卧（伸腿) -> 左侧卧
        5: 4,  # 右侧卧（弯腿) -> 右侧卧
        6: 4,  # 右侧卧（伸腿) -> 右侧卧
        7: 5,  # 坐姿 -> 坐姿
    }

def load_extended_dataset(data_root):
    """加载包含坐姿的扩展数据集"""
    label_mapping = get_extended_label_mapping()
    person_folders = get_person_folders(data_root)
    
    # 还需要处理sit文件夹中的坐姿数据
    sit_folder = os.path.join(data_root, 'sit')
    
    all_data = {}
    
    # 处理每个人员的数据
    for person_name in person_folders:
        person_folder_path = os.path.join(data_root, person_name)
        person_data = {}
        
        # 查找该人员的所有txt文件
        import glob
        txt_files = glob.glob(os.path.join(person_folder_path, f"{person_name}_*.txt"))
        
        for txt_file in txt_files:
            filename = os.path.basename(txt_file)
            try:
                original_label = int(filename.split('_')[-1].split('.')[0])
            except ValueError:
                continue
            
            if original_label not in label_mapping:
                continue
            
            # 加载压力图数据
            frames = load_pressure_data(txt_file)
            if frames is None:
                continue
            
            # 计算平均压力图
            avg_pressure_map = compute_average_pressure_map(frames)
            new_label = label_mapping[original_label]
            
            person_data[new_label] = avg_pressure_map
        
        # 检查sit文件夹中是否有该人员的坐姿数据
        if os.path.exists(sit_folder):
            sit_files = glob.glob(os.path.join(sit_folder, f"{person_name}_*.txt"))
            for sit_file in sit_files:
                frames = load_pressure_data(sit_file)
                if frames is not None:
                    avg_pressure_map = compute_average_pressure_map(frames)
                    person_data[5] = avg_pressure_map  # 标签5为坐姿
                    break  # 只取第一个坐姿文件
        
        if person_data:  # 只有当有数据时才添加
            all_data[person_name] = person_data
    
    return all_data, list(all_data.keys())

# 加载数据
print("正在加载睡姿数据...")
dataset, person_list = load_extended_dataset(data_root)

# 睡姿标签映射
label_names = {
    1: "仰卧",
    2: "俯卧", 
    3: "左侧卧",
    4: "右侧卧",
    5: "坐姿"
}

print(f"数据加载完成！")
print(f"共加载 {len(person_list)} 个用户的数据")
print(f"用户列表: {person_list}")

# 统计各类睡姿数据
posture_stats = {}
for person, data in dataset.items():
    for label in data.keys():
        if label not in posture_stats:
            posture_stats[label] = 0
        posture_stats[label] += 1

print("\n各睡姿数据统计:")
for label, count in sorted(posture_stats.items()):
    print(f"  {label_names[label]}: {count} 个样本")

## 3. 压力图可视化函数定义

In [ ]:
def create_heatmap(pressure_map, title="压力图热力图", figsize=(10, 6)):
    """
    创建压力图热力图
    
    Args:
        pressure_map: 压力图数据 (40, 26)
        title: 图表标题
        figsize: 图表大小
    
    Returns:
        matplotlib figure对象
    """
    fig, ax = plt.subplots(figsize=figsize)
    
    # 创建热力图
    im = ax.imshow(pressure_map, cmap='hot', interpolation='bilinear', aspect='auto')
    
    # 设置标题和标签
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('床垫宽度方向 (传感器列)', fontsize=12)
    ax.set_ylabel('床垫长度方向 (传感器行)', fontsize=12)
    
    # 添加颜色条
    cbar = plt.colorbar(im, ax=ax, shrink=0.8)
    cbar.set_label('压力值', fontsize=12)
    
    # 设置网格
    ax.grid(True, alpha=0.3)
    
    # 美化图表
    plt.tight_layout()
    
    return fig

def plot_pressure_heatmap(person_name, posture_label):
    """
    绘制指定用户和睡姿的压力图热力图
    
    Args:
        person_name: 用户名称
        posture_label: 睡姿标签 (1-5)
    """
    if person_name not in dataset:
        print(f"用户 {person_name} 的数据不存在")
        return
    
    if posture_label not in dataset[person_name]:
        print(f"用户 {person_name} 没有 {label_names[posture_label]} 的数据")
        return
    
    pressure_map = dataset[person_name][posture_label]
    title = f"{person_name} - {label_names[posture_label]} 压力图热力图"
    
    fig = create_heatmap(pressure_map, title)
    
    # 显示统计信息
    max_pressure = np.max(pressure_map)
    min_pressure = np.min(pressure_map)
    mean_pressure = np.mean(pressure_map)
    
    print(f"压力统计信息:")
    print(f"  最大压力值: {max_pressure:.2f}")
    print(f"  最小压力值: {min_pressure:.2f}")
    print(f"  平均压力值: {mean_pressure:.2f}")
    
    plt.show()
    return fig

# 测试函数
if person_list:
    test_person = person_list[0]
    available_postures = list(dataset[test_person].keys())
    if available_postures:
        print(f"\n测试显示用户 {test_person} 的第一个可用睡姿:")
        plot_pressure_heatmap(test_person, available_postures[0])

## 4. Web界面布局设计

In [ ]:
# 创建Web界面的基本样式
style = """
<style>
.sleep-system-title {
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    padding: 20px;
    text-align: center;
    border-radius: 10px;
    margin-bottom: 20px;
    font-family: 'Arial', sans-serif;
    box-shadow: 0 4px 8px rgba(0,0,0,0.1);
}

.control-panel {
    background: #f8f9fa;
    padding: 15px;
    border-radius: 8px;
    margin-bottom: 20px;
    border: 1px solid #dee2e6;
}

.info-panel {
    background: #e3f2fd;
    padding: 10px;
    border-radius: 5px;
    border-left: 4px solid #2196f3;
    margin: 10px 0;
}
</style>
"""

# 显示样式
display(HTML(style))

# 创建标题
title_html = """
<div class="sleep-system-title">
    <h1>🛏️ 睡姿压力图展示系统</h1>
    <p>交互式睡姿压力图热力图可视化平台</p>
</div>
"""

display(HTML(title_html))

# 创建输出区域
output_area = widgets.Output()
info_area = widgets.Output()

print("Web界面布局初始化完成！")

## 5. 用户选择交互组件

In [ ]:
# 创建用户选择下拉菜单
user_dropdown = widgets.Dropdown(
    options=person_list,
    value=person_list[0] if person_list else None,
    description='选择用户:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# 创建睡姿选择按钮组
posture_buttons = {}
posture_colors = {
    1: 'info',     # 仰卧 - 蓝色
    2: 'warning',  # 俯卧 - 橙色
    3: 'success',  # 左侧卧 - 绿色
    4: 'danger',   # 右侧卧 - 红色
    5: 'primary'   # 坐姿 - 深蓝色
}

for label, name in label_names.items():
    button = widgets.Button(
        description=f"{name}",
        button_style=posture_colors[label],
        layout=widgets.Layout(width='120px', height='40px'),
        icon='bed' if label != 5 else 'user'
    )
    posture_buttons[label] = button

# 创建当前选择状态显示
current_selection = widgets.HTML(
    value="<div class='info-panel'><b>当前选择:</b> 请选择用户和睡姿</div>"
)

# 创建用户信息显示
user_info = widgets.HTML(value="")

def update_user_info(user_name):
    """更新用户信息显示"""
    if user_name in dataset:
        available_postures = list(dataset[user_name].keys())
        posture_names = [label_names[p] for p in available_postures]
        
        info_html = f"""
        <div class='info-panel'>
            <b>用户信息:</b> {user_name}<br/>
            <b>可用睡姿:</b> {', '.join(posture_names)} ({len(available_postures)}种)
        </div>
        """
        user_info.value = info_html
        
        # 更新按钮状态
        for label, button in posture_buttons.items():
            if label in available_postures:
                button.disabled = False
                button.tooltip = f"点击查看{label_names[label]}压力图"
            else:
                button.disabled = True
                button.tooltip = f"{user_name}没有{label_names[label]}数据"
    else:
        user_info.value = "<div class='info-panel'>未找到用户数据</div>"

# 初始化用户信息
if person_list:
    update_user_info(person_list[0])

print("用户选择组件创建完成！")

## 6. 睡姿类别展示组件

In [ ]:
# 创建睡姿图例和说明
posture_legend = widgets.HTML(
    value="""
    <div style='background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0;'>
        <h4>📋 睡姿类别说明</h4>
        <div style='display: flex; flex-wrap: wrap; gap: 15px;'>
            <div style='display: flex; align-items: center;'>
                <span style='width: 15px; height: 15px; background: #17a2b8; border-radius: 3px; margin-right: 8px;'></span>
                <span><b>仰卧:</b> 平躺睡姿，脸部朝上</span>
            </div>
            <div style='display: flex; align-items: center;'>
                <span style='width: 15px; height: 15px; background: #ffc107; border-radius: 3px; margin-right: 8px;'></span>
                <span><b>俯卧:</b> 趴着睡姿，脸部朝下</span>
            </div>
            <div style='display: flex; align-items: center;'>
                <span style='width: 15px; height: 15px; background: #28a745; border-radius: 3px; margin-right: 8px;'></span>
                <span><b>左侧卧:</b> 左侧躺睡姿</span>
            </div>
            <div style='display: flex; align-items: center;'>
                <span style='width: 15px; height: 15px; background: #dc3545; border-radius: 3px; margin-right: 8px;'></span>
                <span><b>右侧卧:</b> 右侧躺睡姿</span>
            </div>
            <div style='display: flex; align-items: center;'>
                <span style='width: 15px; height: 15px; background: #007bff; border-radius: 3px; margin-right: 8px;'></span>
                <span><b>坐姿:</b> 床边坐着的姿势</span>
            </div>
        </div>
    </div>
    """
)

# 创建睡姿统计信息显示
def create_statistics_display():
    """创建睡姿统计信息显示"""
    stats_html = "<h4>📊 数据统计</h4><div style='display: flex; flex-wrap: wrap; gap: 10px;'>"
    
    for label, count in sorted(posture_stats.items()):
        color = {
            1: '#17a2b8',
            2: '#ffc107', 
            3: '#28a745',
            4: '#dc3545',
            5: '#007bff'
        }[label]
        
        stats_html += f"""
        <div style='background: {color}; color: white; padding: 10px; border-radius: 5px; text-align: center; min-width: 100px;'>
            <div style='font-size: 18px; font-weight: bold;'>{count}</div>
            <div style='font-size: 12px;'>{label_names[label]}</div>
        </div>
        """
    
    stats_html += "</div>"
    return stats_html

statistics_display = widgets.HTML(
    value=f"""
    <div style='background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0;'>
        {create_statistics_display()}
        <br/>
        <p><b>总用户数:</b> {len(person_list)} | <b>总样本数:</b> {sum(posture_stats.values())}</p>
    </div>
    """
)

print("睡姿类别展示组件创建完成！")

## 7. 热力图生成和更新

In [ ]:
def update_heatmap(user_name, posture_label):
    """
    更新热力图显示
    
    Args:
        user_name: 用户名称
        posture_label: 睡姿标签
    """
    with output_area:
        clear_output(wait=True)
        
        try:
            if user_name not in dataset:
                print(f"❌ 错误: 用户 {user_name} 的数据不存在")
                return
            
            if posture_label not in dataset[user_name]:
                print(f"❌ 错误: 用户 {user_name} 没有 {label_names[posture_label]} 的数据")
                return
            
            # 获取压力图数据
            pressure_map = dataset[user_name][posture_label]
            
            # 创建热力图
            title = f"{user_name} - {label_names[posture_label]} 压力图热力图"
            fig = create_heatmap(pressure_map, title, figsize=(12, 8))
            
            plt.show()
            
            # 显示详细统计信息
            max_pressure = np.max(pressure_map)
            min_pressure = np.min(pressure_map)
            mean_pressure = np.mean(pressure_map)
            std_pressure = np.std(pressure_map)
            
            stats_html = f"""
            <div style='background: #e8f5e8; padding: 15px; border-radius: 8px; margin-top: 10px;'>
                <h4>📈 压力统计信息</h4>
                <div style='display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 10px;'>
                    <div><b>最大压力值:</b> {max_pressure:.2f}</div>
                    <div><b>最小压力值:</b> {min_pressure:.2f}</div>
                    <div><b>平均压力值:</b> {mean_pressure:.2f}</div>
                    <div><b>标准差:</b> {std_pressure:.2f}</div>
                </div>
                <div style='margin-top: 10px;'>
                    <b>数据维度:</b> {pressure_map.shape[0]} × {pressure_map.shape[1]} (行×列)
                </div>
            </div>
            """
            
            display(HTML(stats_html))
            
            # 更新当前选择显示
            current_selection.value = f"""
            <div class='info-panel'>
                <b>当前选择:</b> {user_name} - {label_names[posture_label]} 
                <span style='margin-left: 10px; color: #28a745;'>✅ 数据加载成功</span>
            </div>
            """
            
        except Exception as e:
            print(f"❌ 显示热力图时出错: {str(e)}")
            current_selection.value = f"""
            <div class='info-panel' style='background: #ffe6e6; border-left-color: #dc3545;'>
                <b>错误:</b> 无法显示 {user_name} - {label_names[posture_label]} 的数据
            </div>
            """

def compare_postures(user_name, posture_labels):
    """
    比较多个睡姿的压力图
    
    Args:
        user_name: 用户名称
        posture_labels: 睡姿标签列表
    """
    with output_area:
        clear_output(wait=True)
        
        available_postures = [p for p in posture_labels if p in dataset[user_name]]
        n_postures = len(available_postures)
        
        if n_postures == 0:
            print(f"❌ 用户 {user_name} 没有可比较的睡姿数据")
            return
        
        # 创建子图
        cols = min(3, n_postures)
        rows = (n_postures + cols - 1) // cols
        
        fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
        if n_postures == 1:
            axes = [axes]
        elif rows == 1:
            axes = axes if isinstance(axes, list) else [axes]
        else:
            axes = axes.flatten()
        
        for i, posture_label in enumerate(available_postures):
            pressure_map = dataset[user_name][posture_label]
            
            im = axes[i].imshow(pressure_map, cmap='hot', interpolation='bilinear', aspect='auto')
            axes[i].set_title(f"{label_names[posture_label]}", fontweight='bold')
            axes[i].set_xlabel('床垫宽度')
            axes[i].set_ylabel('床垫长度')
            
            # 添加颜色条
            plt.colorbar(im, ax=axes[i], shrink=0.8)
        
        # 隐藏多余的子图
        for i in range(n_postures, len(axes)):
            axes[i].set_visible(False)
        
        plt.suptitle(f"{user_name} - 睡姿压力图对比", fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()

print("热力图生成和更新函数定义完成！")

## 8. 完整的Web应用集成

In [ ]:
# 定义事件处理函数
def on_user_change(change):
    """用户选择改变时的处理函数"""
    user_name = change['new']
    update_user_info(user_name)
    
    # 清空输出区域
    with output_area:
        clear_output()
        print(f"已选择用户: {user_name}")
        print("请选择要查看的睡姿类别...")

def create_posture_button_handler(posture_label):
    """创建睡姿按钮的事件处理函数"""
    def handler(button):
        user_name = user_dropdown.value
        if user_name:
            update_heatmap(user_name, posture_label)
    return handler

# 绑定事件处理函数
user_dropdown.observe(on_user_change, names='value')

for label, button in posture_buttons.items():
    button.on_click(create_posture_button_handler(label))

# 创建额外功能按钮
compare_button = widgets.Button(
    description="比较所有睡姿",
    button_style='info',
    layout=widgets.Layout(width='150px', height='40px'),
    icon='compare'
)

def on_compare_click(button):
    """比较按钮点击处理函数"""
    user_name = user_dropdown.value
    if user_name and user_name in dataset:
        available_postures = list(dataset[user_name].keys())
        compare_postures(user_name, available_postures)

compare_button.on_click(on_compare_click)

# 创建帮助按钮
help_button = widgets.Button(
    description="使用帮助",
    button_style='secondary',
    layout=widgets.Layout(width='100px', height='40px'),
    icon='question'
)

def show_help(button):
    """显示帮助信息"""
    with output_area:
        clear_output()
        help_html = """
        <div style='background: #f0f8ff; padding: 20px; border-radius: 10px; border: 2px solid #4dabf7;'>
            <h3>🛠️ 使用帮助</h3>
            <ol>
                <li><b>选择用户:</b> 从下拉菜单中选择要查看的用户</li>
                <li><b>选择睡姿:</b> 点击相应的睡姿按钮查看压力图热力图</li>
                <li><b>查看统计:</b> 每个热力图下方都有详细的压力统计信息</li>
                <li><b>比较睡姿:</b> 点击"比较所有睡姿"按钮可以同时查看该用户的所有睡姿</li>
                <li><b>颜色说明:</b> 热力图中红色表示高压力区域，蓝色表示低压力区域</li>
            </ol>
            <h4>🎨 按钮颜色说明:</h4>
            <ul>
                <li><span style='color: #17a2b8;'>蓝色</span> - 仰卧</li>
                <li><span style='color: #ffc107;'>橙色</span> - 俯卧</li>
                <li><span style='color: #28a745;'>绿色</span> - 左侧卧</li>
                <li><span style='color: #dc3545;'>红色</span> - 右侧卧</li>
                <li><span style='color: #007bff;'>深蓝色</span> - 坐姿</li>
            </ul>
        </div>
        """
        display(HTML(help_html))

help_button.on_click(show_help)

# 组装完整的用户界面
control_panel = widgets.VBox([
    widgets.HTML("<div class='control-panel'>"),
    widgets.HBox([user_dropdown, help_button]),
    user_info,
    widgets.HTML("<h4>选择睡姿类别:</h4>"),
    widgets.HBox(list(posture_buttons.values())),
    widgets.HBox([compare_button]),
    current_selection,
    widgets.HTML("</div>")
])

# 创建完整的应用界面
app_interface = widgets.VBox([
    posture_legend,
    statistics_display,
    control_panel,
    output_area
])

# 显示应用界面
display(app_interface)

print("\n🎉 睡姿展示系统已启动！")
print("📝 操作说明:")
print("1. 从下拉菜单选择用户")
print("2. 点击睡姿按钮查看对应的压力图热力图")
print("3. 点击'比较所有睡姿'可同时查看多个睡姿")
print("4. 点击'使用帮助'获取详细使用说明")

# 自动显示第一个用户的第一个可用睡姿（如果存在）
if person_list and person_list[0] in dataset:
    first_user = person_list[0]
    available_postures = list(dataset[first_user].keys())
    if available_postures:
        print(f"\n🔍 自动显示用户 {first_user} 的 {label_names[available_postures[0]]} 睡姿...")
        update_heatmap(first_user, available_postures[0])